In [3]:
#Install Libraries
!pip install pandas requests beautifulsoup4 openpyxl



[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# Import Libraries and Setup
import os
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Make sure the working directory is correct
os.makedirs("extracted_articles", exist_ok=True)

# Hardcoded stopwords
stop_words = {
    'ourselves', 'hers', 'between', 'yourself', 'but', 'again', 'there', 'about', 'once',
    'during', 'out', 'very', 'having', 'with', 'they', 'own', 'an', 'be', 'some', 'for',
    'do', 'its', 'yours', 'such', 'into', 'of', 'most', 'itself', 'other', 'off', 'is',
    's', 'am', 'or', 'who', 'as', 'from', 'him', 'each', 'the', 'themselves', 'until',
    'below', 'are', 'we', 'these', 'your', 'his', 'through', 'don', 'nor', 'me', 'were',
    'her', 'more', 'himself', 'this', 'down', 'should', 'our', 'their', 'while', 'above',
    'both', 'up', 'to', 'ours', 'had', 'she', 'all', 'no', 'when', 'at', 'any', 'before',
    'them', 'same', 'and', 'been', 'have', 'in', 'will', 'on', 'does', 'yourselves', 'then',
    'that', 'because', 'what', 'over', 'why', 'so', 'can', 'did', 'not', 'now', 'under',
    'he', 'you', 'herself', 'has', 'just', 'where', 'too', 'only', 'myself', 'which',
    'those', 'i', 'after', 'few', 'whom', 't', 'being', 'if', 'theirs', 'my', 'against',
    'a', 'by', 'doing', 'it', 'how', 'further', 'was', 'here', 'than'
}


In [5]:
# Define Functions
positive_words = {"good", "happy", "positive", "excellent", "fortunate", "correct", "superior"}
negative_words = {"bad", "sad", "negative", "poor", "unfortunate", "wrong", "inferior"}

def count_syllables(word):
    word = word.lower()
    word = re.sub(r'(es|ed)$', '', word)
    return max(1, len(re.findall(r'[aeiouy]+', word)))

def extract_article(url):
    try:
        res = requests.get(url, timeout=10)
        soup = BeautifulSoup(res.content, 'html.parser')
        title = soup.find('h1') or soup.find('title')
        paragraphs = soup.find_all('p')
        text = ' '.join(p.get_text() for p in paragraphs)
        return (title.get_text(strip=True) if title else ""), text.strip()
    except Exception:
        return "", ""

def simple_sentence_split(text):
    return re.split(r'[.!?]', text)

def simple_word_split(text):
    return re.findall(r'\b\w+\b', text.lower())

def analyze_text(text):
    sentences = [s for s in simple_sentence_split(text) if s.strip()]
    words = simple_word_split(text)
    cleaned_words = [word for word in words if word not in stop_words]
    word_count = len(cleaned_words)
    sent_count = len(sentences)

    positive_score = sum(1 for word in cleaned_words if word in positive_words)
    negative_score = sum(1 for word in cleaned_words if word in negative_words)
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (word_count + 0.000001)

    complex_words = [word for word in cleaned_words if count_syllables(word) > 2]
    complex_word_count = len(complex_words)
    avg_sentence_length = word_count / sent_count if sent_count else 0
    percentage_complex_words = complex_word_count / word_count if word_count else 0
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    avg_words_per_sentence = avg_sentence_length
    syllable_per_word = sum(count_syllables(word) for word in cleaned_words) / word_count if word_count else 0
    avg_word_length = sum(len(word) for word in cleaned_words) / word_count if word_count else 0
    personal_pronouns = len(re.findall(r'\b(I|we|my|ours|us)\b', text, re.I))

    return {
        "POSITIVE SCORE": positive_score,
        "NEGATIVE SCORE": negative_score,
        "POLARITY SCORE": polarity_score,
        "SUBJECTIVITY SCORE": subjectivity_score,
        "AVG SENTENCE LENGTH": avg_sentence_length,
        "PERCENTAGE OF COMPLEX WORDS": percentage_complex_words,
        "FOG INDEX": fog_index,
        "AVG NUMBER OF WORDS PER SENTENCE": avg_words_per_sentence,
        "COMPLEX WORD COUNT": complex_word_count,
        "WORD COUNT": word_count,
        "SYLLABLE PER WORD": syllable_per_word,
        "PERSONAL PRONOUNS": personal_pronouns,
        "AVG WORD LENGTH": avg_word_length,
    }


In [6]:
# Run the Processing
input_df = pd.read_excel("Input.xlsx")
results = []

for _, row in input_df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    title, text = extract_article(url)
    with open(f"extracted_articles/{url_id}.txt", "w", encoding="utf-8") as f:
        f.write(title + "\n\n" + text)
    metrics = analyze_text(text)
    results.append({**row, **metrics})
    print(f"Processed {url_id}")

output_df = pd.DataFrame(results)
output_df.to_excel("Output.xlsx", index=False)
print("✅ Output.xlsx generated successfully!")


Processed Netclan20241017
Processed Netclan20241018
Processed Netclan20241019
Processed Netclan20241020
Processed Netclan20241021
Processed Netclan20241022
Processed Netclan20241023
Processed Netclan20241024
Processed Netclan20241025
Processed Netclan20241026
Processed Netclan20241027
Processed Netclan20241028
Processed Netclan20241029
Processed Netclan20241030
Processed Netclan20241031
Processed Netclan20241032
Processed Netclan20241033
Processed Netclan20241034
Processed Netclan20241035
Processed Netclan20241036
Processed Netclan20241037
Processed Netclan20241038
Processed Netclan20241039
Processed Netclan20241040
Processed Netclan20241041
Processed Netclan20241042
Processed Netclan20241043
Processed Netclan20241044
Processed Netclan20241045
Processed Netclan20241046
Processed Netclan20241047
Processed Netclan20241048
Processed Netclan20241049
Processed Netclan20241050
Processed Netclan20241051
Processed Netclan20241052
Processed Netclan20241053
Processed Netclan20241054
Processed Ne

In [8]:
# Verify Article Extraction
for _, row in input_df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    title, text = extract_article(url)
    
    if not text:
        print(f"⚠️ No text extracted for {url}")
        continue  # Skip if no text found
    
    with open(f"extracted_articles/{url_id}.txt", "w", encoding="utf-8") as f:
        f.write(title + "\n\n" + text)
    
    metrics = analyze_text(text)
    results.append({**row, **metrics})
    print(f"✅ Saved {url_id}.txt")


✅ Saved Netclan20241017.txt
✅ Saved Netclan20241018.txt
✅ Saved Netclan20241019.txt
✅ Saved Netclan20241020.txt
✅ Saved Netclan20241021.txt
✅ Saved Netclan20241022.txt
✅ Saved Netclan20241023.txt
✅ Saved Netclan20241024.txt
✅ Saved Netclan20241025.txt
✅ Saved Netclan20241026.txt
✅ Saved Netclan20241027.txt
✅ Saved Netclan20241028.txt
✅ Saved Netclan20241029.txt
✅ Saved Netclan20241030.txt
✅ Saved Netclan20241031.txt
✅ Saved Netclan20241032.txt
✅ Saved Netclan20241033.txt
✅ Saved Netclan20241034.txt
✅ Saved Netclan20241035.txt
✅ Saved Netclan20241036.txt
✅ Saved Netclan20241037.txt
✅ Saved Netclan20241038.txt
✅ Saved Netclan20241039.txt
✅ Saved Netclan20241040.txt
✅ Saved Netclan20241041.txt
✅ Saved Netclan20241042.txt
✅ Saved Netclan20241043.txt
✅ Saved Netclan20241044.txt
✅ Saved Netclan20241045.txt
✅ Saved Netclan20241046.txt
✅ Saved Netclan20241047.txt
✅ Saved Netclan20241048.txt
✅ Saved Netclan20241049.txt
✅ Saved Netclan20241050.txt
✅ Saved Netclan20241051.txt
✅ Saved Netclan20241